<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para un periodo x. (Agosto - Diciembre 2020)
 

In [273]:
import pandas as pd
import numpy as np

#Gráfico
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [274]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [275]:
data = pd.read_csv('/content/drive/My Drive/Dataset/M_BAJAS.txt',delimiter='~')

In [ ]:
data.head(5)

In [276]:
encabezados= ['Año','Mes','Fecha','Antiguedad','Cant_Bajas']
data.columns=encabezados

In [280]:
data.Fecha = pd.to_datetime(data.Fecha)

In [ ]:
data.head(5)

In [ ]:
data.dtypes

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [ ]:
data.describe(include='all')

In [ ]:
data.info 

In [283]:
#Agrupamos la columna antiguedad
#1 : 1 TRIMESTRE (0-93 DIAS)
#2 : 2 TRIMESTRE (94-186 DIAS)
#3 : 3 TRIMESTRE (187-279 DIAS)
#4 : 4 TRIMESTRE (280-365 DIAS)
#5 : A 2 AÑOS (366-744 DIAS) 
#6 : A 4 AÑOS (745-1488 DIAS)
#7 : Mayor 4 años (Máximo DIAS 17101) 
bins=[0,93,186,279,365,744,1488,17110]
names=['1','2','3','4','5','6','7']
data['Antiguedad'] = pd.cut(data['Antiguedad'],bins,labels=names)

In [ ]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

In [ ]:
data.dropna(subset = ['Antiguedad'] , axis=0, inplace=True)
data['Antiguedad'].astype(str).astype(int)

In [253]:
data=data.drop(['Fecha'],axis=1)

In [271]:
data3= data.groupby(pd.Grouper(key='Antiguedad')).sum()

In [ ]:
data3

In [183]:
data= pd.get_dummies(data,columns=['Antiguedad'])

In [289]:
data

,Año,Mes,Fecha,Antiguedad,Cant_Bajas
0,2019,11,2019-01-11,6,3
1,2020,4,2020-01-04,6,1
2,2019,11,2019-05-11,7,1
3,2020,2,2020-01-02,6,1
4,2019,8,2019-01-08,6,1
...,...,...,...,...,...
17096,2019,8,2019-01-08,6,1
17097,2019,4,2019-01-04,3,1
17098,2019,11,2019-01-11,5,1
17099,2020,7,2020-01-07,6,1


In [293]:
data.corr(method ='pearson')

,Año,Mes,Cant_Bajas
Año,1.000000,-0.318232,0.029800
Mes,-0.318232,1.000000,-0.003926
Cant_Bajas,0.029800,-0.003926,1.000000


In [140]:
x= np.array(data.drop(['Cant_Bajas'],axis=1))
y= np.array(data['Cant_Bajas'])

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
from sklearn.svm import SVR

svr_ = SVR(Kernel='linear',C=1.0,epsilon=0.2)
svr_.fit(x_train, y_train)
y_pred= svr_.predict(x_test)

In [20]:
type(x)

pandas.core.frame.DataFrame

In [21]:
type(y)

pandas.core.series.Series

In [22]:
x

,Año,Mes,Fecha,Antiguedad
0,2019,11,01/11/2019,6
1,2020,4,01/04/2020,6
2,2019,11,05/11/2019,7
3,2020,2,01/02/2020,6
4,2019,8,01/08/2019,6
...,...,...,...,...
17096,2019,8,01/08/2019,6
17097,2019,4,01/04/2019,3
17098,2019,11,01/11/2019,5
17099,2020,7,01/07/2020,6


In [23]:
y

0        3
1        1
2        1
3        1
4        1
        ..
17096    1
17097    1
17098    1
17099    1
17100    1
Name: Cant_Bajas, Length: 17101, dtype: int64